In [23]:
# Feature Extraction Notebook Code

import os
import sys

# Add the path to the src directory
src_path = os.path.join(os.path.dirname(os.path.abspath("__file__")), '..', 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# Import and run the main feature extraction function
from features.extract_features import main as extract_features_main

# Run the feature extraction
extract_features_main()



Processing file: S01_B_RWEO_PreOL_stft.csv
Processing file: S01_C_OLoop_stft.csv
Processing file: S01_D_RWEO_PstOL_stft.csv
Processing file: S01_E_RWEO_PreCL_stft.csv
Processing file: S01_F_CL_Sil_50_100_stft.csv
Processing file: S01_G_RWEO_PstCL_stft.csv
Processing file: S02_B_RWEO_PreOL_stft.csv
Processing file: S02_C_OLoop_stft.csv
Processing file: S02_D_RWEO_PstOL_stft.csv
Processing file: S02_E_RWEO_PreCL_stft.csv
Processing file: S02_F_CL_Sil_50_100_stft.csv
Processing file: S02_G_RWEO_PstCL_stft.csv
Processing file: S03_B_RWEO_PreOL_stft.csv
Processing file: S03_C_OLoop_stft.csv
Processing file: S03_D_RWEO_PstOL_stft.csv
Processing file: S03_E_RWEO_PreCL_stft.csv
Processing file: S03_F_CL_Sil_50_100_stft.csv
Processing file: S03_G_RWEO_PstCL_stft.csv
Processing file: S04_B_RWEO_PreOL_stft.csv
Processing file: S04_C_OLoop_stft.csv
Processing file: S04_D_RWEO_PstOL_stft.csv
Processing file: S04_F_CL_Sil_50_100_stft.csv
Processing file: S04_G_RWEO_PstCL_stft.csv
Processing file: S0